## 写在前面

- 魔法方法__标识__：双下划线开头和结尾
- \_\_init__：构造函数，初始化对象的状态
- \_\_delinit__：析构函数
- super
- 类似序列和映射
    - \_\_len__
    - \_\_getitem__
    - \_\_setitem__
    - \_\_delitem__
- 特性
    - property
    - staticmethod
    - classmethod
    - 魔法方法和特性
        - \_\_getattr\_\_
        - \_\_setattr\_\_
        - \_\_delattr\_\_
        - \_\_getattribute\_\_
- 迭代器
    - \_\_iter\_\_
    - \_\_next\_\_
    - iter函数
    - next函数
- 生成器
    - send
    - throw
    - close

## 重写构造函数

### 使用未关联的超类调用构造函数  
新式类、旧式类都可以用

In [7]:
class Filter:
    def __init(self):
        self.block = []
    def filt(self, sequence):
        return [x for x in sequence if x not in self.block]

class FruitFilter(Filter):
    def __init(self):
        Filter.__init__(self)
        self.out = []

In [8]:
ff = FruitFilter()
fruits = ['melon', 'banaba', 'apple', 'peach', 'lemon', 'straberry']
ff.block = ['lemon']
ff.filt(fruits)

['melon', 'banaba', 'apple', 'peach', 'straberry']

### super函数  
不能用于旧式类

In [13]:
class Filter:
    def __init__(self):
        self.block = []
    def filt(self, sequence):
        return [x for x in sequence if x not in self.block]

class NameFilter(Filter):
    def __init__(self):
        super().__init__()
        self.names = []

In [14]:
names = ['David', 'Goslin', 'Tompson', 'Person', 'Trump']
nf = NameFilter()
nf.block = ['David']
nf.filt(names)

['Goslin', 'Tompson', 'Person', 'Trump']

## 创建类似于序列和映射的对象

### 使用魔法方法创建

In [162]:
# 创建等差数列的字典
class MyDict:
    def __init__(self, sequence={}):
        self.sequence = sequence
    
    def __len__(self):
        return len(self.sequence)
    
    def __getitem__(self, key):
        try:
            return self.sequence[key]
        except:
            raise
    
    def __setitem__(self, key, value):
        self.sequence[key] = value
    
    def __delitem__(self, key):
        del self.sequence[key]

In [163]:
info = {"name":"Goslin", "age":18, "sex":"Male"}
s = MyDict(info)  # 实例化一个字典对象

In [164]:
s["name"]  # 调用了__getitem__

'Goslin'

In [165]:
s["hight"] = 180  # 调用了__setitem__

In [166]:
len(s)  # 调用了__len__

4

In [167]:
del s["hight"]  # 调用了__delitem__

In [168]:
s["hight"]

KeyError: 'hight'

### 继承：由list、dict、str派生

In [82]:
class CounterList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.counter = 0
    def __getitem__(self, index):  # 并不一定会捕捉到所有访问列表的操作，如pop函数
        self.counter += 1
#         return self.counter
        return super(CounterList, self).__getitem__(index)  # 在当前类和当前实例作为super的参数时，调用的是超类的方法

In [73]:
c1 = CounterList(range(10))

In [74]:
c1

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [75]:
c1.reverse()
c1

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [76]:
del c1[3:6]
c1

[9, 8, 7, 3, 2, 1, 0]

In [77]:
c1.counter

0

In [78]:
c1[2] + c1[4]

9

In [79]:
c1.counter

2

In [80]:
c1.pop()

0

In [81]:
c1.counter  # __getitem__没有捕捉到pop函数的访问操作

2

## 特性  
- 定义：通过存取器方法定义的属性  
- 来源：不可能为所有的属性提供存取器方法  
- 对于新式类，使用特性，而不是一直使用存取器方法
- 调用：property(fget, fset, fdel, doc)，将存取方法作为参数
- 参数解释：
    - 如果没有指定任何参数，创建的特性将既不可读也不可写
    - 如果只指定一个参数（获取方法），创建的特性将是只读的
    - 第三个参数是可选的，指定用于删除属性的方法（这个方法不接受任何参数）
    - 第四个参数也是可选的，指定一个文档字符串
- 注意：
    - property是一个类，通过魔法方法完成读、写、删、文档的操作，\_\_get\_\_、\_\_set\_\_、\_\_delete__

### property函数

In [83]:
# 没有使用特性，需要通过存取器方法调用方法中定义的属性
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def get_size(self):
        return self.width, self.height
    def set_size(self, size):
        self.width, self.height = size

In [84]:
r = Rectangle()

In [85]:
r.width = 50
r.height = 100
r.get_size()

(50, 100)

In [86]:
r.set_size((100, 50))
r.get_size()

(100, 50)

In [106]:
# 使用特性，直接获取存取器中定义的属性
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def get_size(self):
        'This is a get_size docstring.'
        return self.width, self.height
    def set_size(self, size):  # size是通过存取器方法定义的属性
        self.width, self.height = size
    def del_size(self):
        del self.width, self.height
    size = property(get_size, set_size, del_size, 'This is a property docstring.')

In [107]:
r = Rectangle()

In [108]:
r.size

(0, 0)

In [109]:
r.size = (10, 20)
r.size

(10, 20)

In [110]:
del r.width

In [114]:
r.height

20

- py2中使用property
<code>
class Score:
    def \_\_init__(self, score):
        self.score = score
    @property
    def score(self):
        return self.score
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise TypeError("The value must be int.")
        if (value > 100) | (value < 0):
            raise ValueError("The value 0~100.")
        self.score = value
    @score.deleter
    def score(self):
        del self.score
</code>

### 静态方法和类方法  
- 静态方法：定义的时候不需要self参数，实现方式和property函数类似，可直接通过类来调用
- 类方法：定义的时候需要类似于self的参数，通常命名为cls，实现方式和property函数类似，可直接通过类来调用，参数cls将自动关联到类

#### 类似property函数的定义

In [12]:
class MyClass:
    def smeth():
        print("This is a static method.")
    smeth = staticmethod(smeth)
    def cmeth(cls):
        print("This is a class method.")
    cmeth = classmethod(cmeth)

In [13]:
MyClass.smeth()

This is a static method.


In [14]:
MyClass.cmeth()

This is a class method.


#### 通过修饰器定义

In [15]:
class MyClass:
    @staticmethod
    def smeth():
        print("This is a static method.")
    @classmethod
    def cmeth(cls):
        print("This is a class method.")

In [16]:
MyClass.smeth()

This is a static method.


In [17]:
MyClass.cmeth()

This is a class method.


### **通过魔法方法定义**
- \_\_getattribute__(self, name) ：在属性被访问时自动调用（只适用于新式类）。
- \_\_getattr__(self, name) ：在属性被访问而对象没有这样的属性时自动调用。
- \_\_setattr__(self, name, value) ：试图给属性赋值时自动调用。
- \_\_delattr__(self, name) ：试图删除属性时自动调用。
- 相比函数 property ，这些魔法方法使用起来要棘手些（从某种程度上说，效率也更低）

In [63]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def __getattr__(self, name):
        if name == "size":
            return self.width, self.height
        else:
            raise AttributeError(eval(name) + " is not defined.")
    def __setattr__(self, name, value):
        if name == "size":
            self.width, self.height = value
        else:
            self.__dict__[name] = value
    def __delattr__(self, name):
        if name == "size":
            del self.width, self.height
        else:
            raise AttributeError(eval(name) + " is not defined.")

In [64]:
r = Rectangle()

In [65]:
r.name = "Person"

In [66]:
r.name

'Person'

In [67]:
r.size = (100, 200)

In [68]:
r.__dict__

{'height': 200, 'name': 'Person', 'width': 100}

In [69]:
del r.size

AssertionError: 

## 迭代器

- 方法\_\_iter\_\_返回一个迭代器，它是包含方法\_\_next\_\_的对象，而调用这个方法时可不提供任何参数。当你调用方法\_\_next\_\_时，迭代器应返回其下一个值。如果迭代器没有可供返回的值，应引发StopIteration异常
- 正规定义：
    - 仅仅实现了方法\_\_iter\_\_ 的对象是可迭代的
    - 同时实现了方法\_\_next\_\_的对象是迭代器
    - list是可迭代的，但不是迭代器

### 可迭代对象  
对可迭代对象使用iter函数，获得一个迭代器

In [121]:
lst = iter([1, 2, 3, 4])

In [122]:
next(lst)

1

In [123]:
next(lst)

2

### 斐波那契数列迭代器

In [17]:
class Fibs:
    def __init__(self):
        self.a = 0
        self.b = 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a

In [18]:
f = Fibs()

In [19]:
for x in f:
    if x > 1000:
        print(x)
        break

1597


### 从迭代器创建序列

In [1]:
class Fibs:
    def __init__(self, index):
        self.a = -1
        self.b = 0
        self.n = 0
        self.index = index
    def __iter__(self):
        return self
    def __next__(self):
        self.n += 1
        self.a, self.b = self.b, abs(self.a)+self.b
        if self.n > self.index:
            raise StopIteration
        return self.a

In [7]:
f = Fibs(10)
list(f)[-1]

34

In [108]:
class MyRange:
    def __init__(self, start=0, stop=1, step=1):
        self.value = start-step
        self.stop = stop
        self.step = step
    def __iter__(self):
        return self
    def __next__(self):
        self.value += self.step
        if self.value >= self.stop:
            raise StopIteration
        return self.value

In [116]:
rg = MyRange(0, 10, 1)

In [117]:
list(rg)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## 生成器  
- 生成器是一种使用普通函数语法定义的迭代器
- 包含yield语句的函数都是生成器
- 与普通函数的区别：
    - 生成器一次调用，可以多次返回值，每次一个
    - 普通函数一次调用，return只能返回一次值

### 创建生成器
- 列表推导是一种从其他列表创建列表的方式
- 生成器推导工作原理与列表推导相同，但返回的是一个生成器
- 如果产生大量的值，列表推导会立即实例化一个列表，而生成器每次返回一个值

In [3]:
def flatten(nested):
    for sublist in nested:
        for element in sublist:
            yield element

In [5]:
nested = [[1,2], [3,4], [5]]
for n in flatten(nested):
    print(n, end=' ')

1 2 3 4 5 

### 简单生成器

In [14]:
g = (i**2 for i in range(10))  # 圆括号

In [15]:
next(g)

0

### 递归式生成器

In [6]:
def flatten(nested):
    try:
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

In [7]:
nested = [1,2,3,[4,5],[6,[7,8,9]]]
for n in flatten(nested):
    print(n, end=' ')

1 2 3 4 5 6 7 8 9 

In [23]:
def flatten(nested):
    try:
        try:
            nested + ''
        except TypeError:
            pass
        else:
            raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

In [25]:
nested = [1,2,3,[4,5],[6,[7,8,9], 'hello', 'world']]
for element in flatten(nested):
    print(element, end=' ')

1 2 3 4 5 6 7 8 9 hello world 

### 生成器的方法

#### send  
- 仅在生成器挂起时(即第一次执行yield之后)，使用send才有意义
- 要在此之前向生成器提供信息，可使用生成器的函数的参数
- 如果一定要在生成器刚启动时对其调用方法send，可向它传递参数None 。

In [36]:
def repeater(value):
    while True:
        new = (yield value)
        if new is not None:
            value = new

In [37]:
r1 = repeater(42)

In [38]:
next(r1)

42

In [39]:
r1.send("hello world")

'hello world'

In [40]:
r2 = repeater(None)
next(r)

In [41]:
r2.send("hello world")

'hello world'

#### throw

#### close

### 模拟生成器

In [43]:
def flatten(nested):
    result = []
    try:
        # 不迭代类似于字符串的对象：
        try: 
            nested + ''
        except TypeError: 
            pass
        else: 
            raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                result.append(element)
    except TypeError:
        result.append(nested)
    return result

## 八皇后  
你需要将8个皇后放在棋盘上，条件是任何一个皇后都不能威胁其他皇后，即任何两个皇后都不能吃掉对方